# 과제 소개
* 중요한 데이터가 저장된 USB를 습득
* 해당 USB에는 2009년에 생성된 파일들만 존재
* 분석결과 현재는 사용되지 않는 오래된 USB 보안 도구를 이용하여 파일을 저장
* 윈도우 XP에서만 동작 가능

# ◎ 과제 개요
## 1. 메뉴얼로부터 보안 도구 및 기능 그리고 암호화 방식 확인.
* 보안 도구 및 기능 : USBSafe, 암호화 영역 생성 및 관리
* 암호화 방식 : <span style = 'color:red'> ***128Bit SEED*** </span>

## 2. 암호화 영역 hex로부터 운용모드 추측.
* 암호화 영역 : 보안영역(**SecuArea.img**), 뱅크매니저(**BankMan.img**)으로 추측
* 같은 값이 많이 나옴 $\to$ <span style = 'color:red'> ***ECB*** </span> 예상 

## 3. IDA로 USBSafe.exe 분석
### 3.1 SEED Sbox로 부터 암호화, 복호화, 키 생성 함수 추정

### 3.2 USBSafeu.cfg 분석
* 랜덤 64바이트 생성한 후, **비밀키**뿐만 아니라 상수(<span style = 'color:red'> **JiranSoft_Win32** </span>) 로 암호화하여 cfg파일에 저장한 것을 확인
* 이를 통해, <span style = 'color:red'>비밀키 없이 랜덤 64바이트 복원</span> $\to$ 상수로 암호화한 값을 저장한 것이 <span style = 'color:red'> 취약점 </span>


### 3.3 SecuArea.img 분석
+ 위에서 생성한 <span style = 'color:red'> **랜덤 64바이트**</span>로 암호화 하는 것을 확인 (정확하게는 64바이트 중 앞에 16바이트)
+ 이를 통해 SecuArea.img 복원
+ FAT32 파일 시스템 확인 후, ftk_imager로 복구하여 password 확인

# ◎ 과제 세부 내용

## 1. 사용자 메뉴얼
* USB 보안 도구 : **USBsafe** 라는 프로그램
* **주요 기능** ( 사용자 메뉴얼 기반 )
    1. 일반 메모리와 차별화된 **암호화 영역** ( 보안 영역, 노트 패드, 뱅크매니저 등등 )
    2. **128Bit SEED** 암호화 방식 사용
<img src = "사진/주요기능및특장점.png" width = "800px;">

* 실제 실행 화면
    - 보안 영역, 보안 노트, 뱅크매니저, 환경설정 시 pw 요구
<img src = '사진/실행화면.png'>

## 2. 암호화 영역 hex로 운용모드 확인
* **SecuArea.img**
    - 크기도 딱 5MB이면서 이름도 보안영역이므로 **사용자가 생성한 5MB의 보안 영역**으로 추측
    - 같은 16byte(SEED의 블록크기)가 많이 중복되어 나옴 $\to$ **ECB 운용모드** 사용한 것으로 추측


* **SecuArea.img 의 hex**
    <img src = '사진/SecuArea_hex.png'>

## 3. IDA로 USBSafe.exe 분석

### 3.1 SEED Sbox로 암복호화, 키 생성 함수 추정
     1. SEED의 Sbox의 값 중 하나( 0x2989a1a8 ) 검색 ( ※ 리틀 엔디안 이므로 a8 a1 89 29 검색 )
        + **검색 결과** (뒤에 값들도 다 맞는 것을 확인 $\to$ **SEED 암호화** 실제로 사용한 것 확인)
<img src = "사진/Sbox검색.png">

    2. 위 데이터 부분을 참조하는 함수 3개
        * sub_415D00 (SEED_KEYSET_CORE) : 8byte 짜리 16개를 저장하는 것으로 보아 16개의 라운드 키 저장하는 함수로 추측
        * sub_414BF0 (SEED_DEC) : 서브키를 뒤에서부터 참조하는 것으로 보아 복호화 함수로 추측
        * sub_413AE0 (SEED_ENC) : 서브키를 앞에서부터 참조하는 것으로 보아 암호화 함수로 추측



### 3.2 USBSafeu.cfg 분석
1. USBSAFE_KEY_gen 함수에서  <span style = 'color:red'> **0x1C8 정도의 바이트** </span>를 Buffer에 저장하고 이를 파일에 저장하므로  <span style = 'color:red'> **"USBSafeu.cfg" 생성 추측** </span>
2. 버퍼에 0xCC를 xor하여 파일에 저장
3. USBSAFE_KEY_gen 이후 Buffer[:20] = "JrUS00...00"와 Buffer[0x140:180], Buffer[0x181:0x1C1]엔 암호화된 랜덤값이 저장됨
4. Buffer[0x181:0x1C1]는  <span style = 'color:red'> **"JiranSoft_Win32"** </span>로 암호화됨
**USBSafeu.cfg 생성 함수**
<img src = "사진/usbsafe_cfg_gen.png">

**USBSAFE_KEY_gen 함수**
<img src = "사진/usbsafe_key_gen.png">

# SEED 암호화 함수
### cryptography 모듈 사용
$$ Test \; vector $$
<img src = '사진/SEED_testvector.png'>

In [1]:
import cryptography.fernet as fn

In [2]:
def seedcipher(key):
    ret = fn.Cipher(fn.algorithms.SEED(key), fn.modes.ECB())
    return ret

def dec(key, ciphertexts):
    cipher = seedcipher(key)
    return cipher.decryptor().update(ciphertexts)

In [3]:
testk = bytes([0 for x in range(16)])
testp = bytes([x for x in range(16)])
testk, testp

(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f')

In [4]:
testc = seedcipher(testk).encryptor().update(testp+ testp)
testc[:16].hex().upper(), testc[16:].hex().upper()

('5EBAC6E0054E166819AFF1CC6D346CDB', '5EBAC6E0054E166819AFF1CC6D346CDB')

# cfg 복호화 코드

In [5]:
file_path = "./USB_SYSTEM/"
file_name = "USBSafeu.cfg"
with open(file_path + file_name, 'rb') as f:
    cfgfile = f.read()
newcfgfile = bytes([cfgfile[x] ^ 0xCC for x in range(len(cfgfile))]) # 0xCC XOR 해서 바이트로 읽기
newcfgfile[:0x20]

b'JrUS\x00\x00\xd6\x02\x00\x00\x00\x00\xaa\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [6]:
key = b'JiranSoft_Win32' + b'\x00'
cfgrandom = dec(key, newcfgfile[0x181 : 0x1C1]) # 상수 키로 암호화한 부분 복호화 하기
cfgrandom, cfgrandom[0x32:0x32+11]

(b'OCDVLIBDJNLFSTIOZVTPSPCIUBZEIKOSKYIIXBKIVOJHGLBYWMUSBSAFE_KEYDJP',
 b'USBSAFE_KEY')

In [7]:
cfg_decryption = newcfgfile[:0x140] + cfgrandom + newcfgfile[0x180:0x181] + cfgrandom + newcfgfile[0x1C1:]
with open("cfgdec", 'wb') as f:
    f.write(cfg_decryption)

### 3.3 SecuArea.img 복원
1. USBSAFE_SECUAREA_IMAGE_HEADER1 + **80 바이트**를 저장하므로 SecuArea.img의 헤더 부분 생성으로 추측
2. v6의 80 바이트를 Buffer를 사용하여 암호화
3. Buffer는 USBSAFE_KEY_check 이후, cfg에서 복호화한 64 바이트가 저장됨 (cfg에서 복호화한 것을 사용)
4. 3840(0xf00) 바이트 이후 Buffer의 16바이트를 "USBSAFE_JIRANSOFT"로 암호화하여 저장
<img src = "사진/secuarea_header_enc.png">

# SecuArea.img의 복호화 코드
1. header(0x0 ~ 0x80) : cfg에서 복원한 바이트로 복원
2. 0xf00 ~ 0xf10 : USBSAFE_JIRANSOFT로 복원
3. 0x1000 이후 : cfg에서 복원한 바이트로 복원

In [8]:
Buffer = cfgrandom
key = Buffer[:16]
file_name = "SecuArea.img"
with open(file_path + file_name, 'rb') as f:
    imgfile = f.read()
header_decyption = dec(key, imgfile[:0x80])
img_decryption = dec(key, imgfile)
header_decyption

b'USBSAFE_SECUAREA_IMAGE_HEADER1USBSAFE_SECUAREA_IMAGE_HEADER1USBSAFE_SECUAREA_IMAGE_HEADER1USBSAFE_SECUAREA_IMAGE_HEADER1\x00\x00\x00\x00\x00\x00\x00\x00'

In [9]:
key = b'USBSAFE_JIRANSOFT'[:16]
decyption3840 = dec(key, imgfile[3840:3840+16])
decyption3840, Buffer[:16]

(b'OCDVLIBDJNLF\x00\x00\x00\x00', b'OCDVLIBDJNLFSTIO')

In [10]:
with open("SecuArea_full_decryption", 'wb') as f:
    f.write(header_decyption + imgfile[0x80:0xF00] + decyption3840 + imgfile[0xF10:0x1000] + img_decryption[0x1000:])

In [11]:
with open("SecuArea_dec.img", 'wb') as f:
    f.write(img_decryption[0x1000:])

## 복호화 후
    * 파일 포맷이 나옴
    * 읽을 수 있는 단어가 나옴
    * 내부에 password.txt 파일 흔적
<img src = '사진/복호화.png'>
<img src = '사진/password.png'>

### 0x1000 이후의 파일로 저장 후, ftk_image로 이미지 복원
<img src = '사진/ftk복원.png'>